In [140]:
import os 
import cv2
import pandas as pd 
import math
from collections import defaultdict
import inflection as inf

In [128]:
folder = "realNotFruit"
prefix = "diffobj"
resized_img_dir = f"{prefix}_resized_images"
duplicate_images_dir = f"{prefix}_duplicate_images"
det_ann_filename = "diffobject_ann.csv"

In [129]:

dim = 256

images = os.listdir(folder)

name_map = {}
coco_name_map = {}

if not os.path.exists(resized_img_dir):
    os.mkdir(resized_img_dir)

for i, image in enumerate(images):
    img = cv2.imread(folder + "/" + image)
    h, w, _ = img.shape
    max_dim = max(h, w)
    pad_h = (max_dim - h) // 2
    pad_w = (max_dim - w) // 2
    padded_img = cv2.copyMakeBorder(img, pad_h, pad_h, pad_w, pad_w, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    resized_img = cv2.resize(padded_img, (dim, dim))
    name_map[str(i).zfill(6) + ".jpg"] = image
    coco_name_map[image] = str(i).zfill(6) + ".jpg"
    cv2.imwrite(resized_img_dir + "/" + str(i).zfill(6) + ".jpg", resized_img)


### Format determiner annotation

In [130]:
ann_dir = "./"

df = pd.read_csv(os.path.join(ann_dir, det_ann_filename))
pad = 12 
print(df)

counts = {}
captions_map = defaultdict(list)

for i, row in df.iterrows():
    if not math.isnan(row["coco_id"]): 
        coco_filename = str(int(row["coco_id"])).zfill(pad) + ".jpg"
        image_file = coco_name_map[coco_filename]
        captions = row["captions"].split(",")
        captions_map[image_file] = captions
    else: 
        coco_filename = None



    coco_id  counts                                           captions
0        63       8  a person, a baseball bat, a sport ball, every ...
1       968       7  a cat, some cats, all cats, any cat, either ca...
2      1385       8  a bird, all birds, those birds, these birds, f...
3      1730       8  a person, a sport ball, a baseball glove, all ...
4      2427      11  a person,all persons, every person, each perso...
5      6069       6  a bird, all birds, those birds, these birds, e...
6      6947       9  a person, all persons, every person, each pers...
7      7754       4              a person, a cat, a keyboard, a laptop
8    261972       9  a person, a horse, all horses, any horse, ever...
9    262186       8  a cat, a bench, some cats, all cats, any cat, ...
10   262291       3                       a person, a stop sign, a hat


### duplicate images



In [131]:
df = pd.DataFrame(columns=["image", "coco_filename", "count"])
df["image"] = os.listdir("resized_images")
df["coco_filename"] = df["image"].map(name_map)
df["count"] = 1

df.to_csv(f"{prefix}_counts_template.csv", index=False)

In [132]:
captions_map["000002.jpg"]

['a bird',
 ' all birds',
 ' those birds',
 ' these birds',
 ' few birds',
 ' every bird',
 ' each bird',
 ' several birds']

In [133]:
df = pd.read_csv(f"{prefix}_counts_template.csv")
df["count"] = df.apply(lambda row: len(captions_map[row["image"]]), axis=1)
print(df)
df.to_csv(f"{prefix}_counts.csv", index=False)

         image     coco_filename  count
0   000000.jpg  000000000063.jpg      8
1   000001.jpg  000000000968.jpg      7
2   000002.jpg  000000001385.jpg      8
3   000003.jpg  000000001730.jpg      8
4   000004.jpg  000000002427.jpg     11
5   000005.jpg  000000006069.jpg      6
6   000006.jpg  000000006947.jpg      9
7   000007.jpg  000000007101.jpg      0
8   000008.jpg  000000007754.jpg      4
9   000009.jpg  000000008221.jpg      0
10  000010.jpg  000000261972.jpg      9
11  000011.jpg  000000262186.jpg      8
12  000012.jpg  000000262291.jpg      3
13  000013.jpg  000000262567.jpg      0
14  000014.jpg  000000262593.jpg      0
15  000015.jpg  000000262615.jpg      0
16  000016.jpg  000000262647.jpg      0
17  000017.jpg  000000262730.jpg      0
18  000018.jpg  000000262920.jpg      0
19  000019.jpg  000000262982.jpg      0
20  000020.jpg  000000263059.jpg      0
21  000021.jpg  000000263217.jpg      0
22  000022.jpg  000000263337.jpg      0
23  000023.jpg  000000264368.jpg      0


In [134]:
if not os.path.exists(duplicate_images_dir): 
    os.mkdir(duplicate_images_dir)

import pandas as pd
import shutil

df = pd.read_csv(f"{prefix}_counts.csv")

image_filenames = []
image_captions = []

for i, row in df.iterrows():
    image = row["image"]
    count = row["count"]
    for j in range(count):
        shutil.copy(resized_img_dir + "/" + image, duplicate_images_dir + "/" + str(i).zfill(6) + "-" + str(j).zfill(2) + ".jpg")
        image_filenames.append(str(i).zfill(6) + "-" + str(j).zfill(2) + ".jpg") 
        image_captions.append(captions_map[image][j])




In [135]:
df = pd.DataFrame(columns=["image", "caption"])
df["image"] = image_filenames
df["caption"] = image_captions
df.to_csv(f"{prefix}_annotations.csv", index=False)

PermissionError: [Errno 13] Permission denied: 'diffobj_annotations.csv'

###Merge

In [4]:
# merge the annotations 

import os 
import cv2
import pandas as pd 
import math
from collections import defaultdict
import inflection as inf
import json

ann_dir = "./annotations/smallv1"
objdet_filename = "iccv-real-small-objdet.json"
det_filename = "iccv-real-small-detlabels.json"
orig_annotations_filepath = "./annotations/annotations_val.json"

objdet_json = json.load(open(os.path.join(ann_dir, objdet_filename)))
det_json = json.load(open(os.path.join(ann_dir, det_filename)))
orig_json = json.load(open(orig_annotations_filepath))

categories = orig_json["categories"]
orig_cat_map = {}

for cat in categories:
    orig_cat_map[cat["name"]] = cat["id"]
print(categories)

images = []
objdet_images = objdet_json["images"]
det_images = det_json["images"]
det_categories = det_json["categories"]
oracle_annotations = objdet_json["annotations"]

cat_map = {}
id_map = {}

for i, cat in enumerate(det_categories):
    cat_map[cat["id"]] = cat["name"]

for i, img in enumerate(det_images): 
    img = img.copy() 
    image_id = img["id"]
    img["file_name"] = "images/" + img["file_name"].split("-")[0] + ".jpg"
    id_map[image_id] = i 
    images.append(img)

annotations = []
for ann in det_json["annotations"]:
    i = id_map[ann["image_id"]]
    caption = cat_map[ann["category_id"]]
    if "juice" in caption:
        caption = caption.split()[0] + " papaya juice"
    images[i]["caption"] = caption
    name = inf.singularize(" ".join(caption.split()[1:]))
    ann["category_id"] = orig_cat_map[name]
    annotations.append(ann)

new_images = []
for i, image in enumerate(images): 
    if image.get("caption"):
        new_images.append(image)

    
new_annotations = {
    "images": new_images,
    "annotations": annotations, 
    "input_oracle_annotations": oracle_annotations,
    "categories": categories
}

json.dump(new_annotations, open(os.path.join(ann_dir, "merged.json"), "w"))

[{'id': 0, 'name': 'apple', 'supercategory': 'countable'}, {'id': 1, 'name': 'onion', 'supercategory': 'countable'}, {'id': 2, 'name': 'avocado', 'supercategory': 'countable'}, {'id': 3, 'name': 'orange', 'supercategory': 'countable'}, {'id': 4, 'name': 'egg', 'supercategory': 'countable'}, {'id': 5, 'name': 'carrot', 'supercategory': 'countable'}, {'id': 6, 'name': 'cucumber', 'supercategory': 'countable'}, {'id': 7, 'name': 'lemon', 'supercategory': 'countable'}, {'id': 8, 'name': 'garlic', 'supercategory': 'countable'}, {'id': 9, 'name': 'banana', 'supercategory': 'countable'}, {'id': 10, 'name': 'grape juice', 'supercategory': 'uncountable_liquid'}, {'id': 11, 'name': 'cranberry juice', 'supercategory': 'uncountable_liquid'}, {'id': 12, 'name': 'blueberry juice', 'supercategory': 'uncountable_liquid'}, {'id': 13, 'name': 'papaya juice', 'supercategory': 'uncountable_liquid'}, {'id': 14, 'name': 'vegetable juice', 'supercategory': 'uncountable_liquid'}, {'id': 15, 'name': 'tray', 's